# TF-IDF with ngram

## Simple

In [ ]:
import pandas as pd
import numpy as np
import re
import warnings
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import math
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings("ignore")

nltk.download('punkt')
nltk.download('punkt_tab')

lyrics_df = pd.read_excel('/content/databersih.xlsx')
ground_truth_df = pd.read_excel('/content/GT UAS NLP_simple.xlsx')

stemmer = PorterStemmer()
remove_words = ["song", "with", "lyrics", "from", "the", "album", "released", "in", "before", "after", "since", "s"]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
def clean_lyrics(text):
    if isinstance(text, str):
        text = re.sub(r'\[.*?\]', '', text)
        text = re.sub(r'\(.*?\)', '', text)
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)
        text = text.lower()
        tokens = word_tokenize(text)
        tokens = [stemmer.stem(word) for word in tokens if word not in remove_words]
        text = ' '.join(tokens)
    return text

lyrics_df['Processed_Lyrics'] = lyrics_df['lyrics'].apply(clean_lyrics)

print("\nCleaned Lyrics Sample:")
print(lyrics_df[['title', 'Processed_Lyrics']].head())

def parse_song_ids(song_id_entry):
    if pd.isnull(song_id_entry):
        return []
    if isinstance(song_id_entry, int):
        return [song_id_entry]
    if isinstance(song_id_entry, float) and np.isnan(song_id_entry):
        return []
    song_id_str = str(song_id_entry)
    return [int(id_.strip()) for id_ in song_id_str.split(',') if id_.strip().isdigit()]

ground_truth_df['Relevant_Song_IDs'] = ground_truth_df['song_id'].apply(parse_song_ids)
ground_truth_df = ground_truth_df[ground_truth_df['Relevant_Song_IDs'].map(len) > 0].reset_index(drop=True)
print("\nParsed Ground Truth:")
print(ground_truth_df[['query', 'Relevant_Song_IDs', 'total']].head())

def preprocess_query(query):
    if isinstance(query, str):
        query = re.sub(r'[^\w\s]', '', query)
        query = re.sub(r'\d+', '', query)
        query = query.lower()
        tokens = word_tokenize(query)
        tokens = [stemmer.stem(word) for word in tokens if word not in remove_words]
        return ' '.join(tokens)
    return query

ground_truth_df['Processed_Query'] = ground_truth_df['query'].apply(preprocess_query)


Cleaned Lyrics Sample:
                   title                                   Processed_Lyrics
0      Chasing Pavements  ive made up my mind dont need to think it over...
1          Cold Shoulder  you say it all my head and thing i think just ...
2         Hometown Glory  ive been walk same way as i did miss out crack...
3  Make You Feel My Love  when rain is blow your face and whole world is...
4                My Same  aye aye ayeay aye aye ayeay aye aye ayeay aye ...

Parsed Ground Truth:
           query                                  Relevant_Song_IDs  total
0    love you in          [129, 1433, 1977, 2978, 3214, 3320, 4514]      7
1   home tonight  [561, 1019, 1455, 1671, 1686, 2428, 2434, 3126...     12
2  something new  [20, 484, 671, 783, 999, 1463, 2556, 2607, 264...     18
3     i held you                             [44, 1520, 4082, 4503]      4
4  stars tonight                                   [95, 2142, 3177]      3


In [ ]:
def precision_at_k(relevant, retrieved, k):
    if k == 0:
        return 0.0
    retrieved_k = retrieved[:k]
    relevant_retrieved = set(retrieved_k).intersection(relevant)
    return len(relevant_retrieved) / k

def recall_at_k(relevant, retrieved, k):
    if len(relevant) == 0:
        return 0.0
    retrieved_k = retrieved[:k]
    relevant_retrieved = set(retrieved_k).intersection(relevant)
    return len(relevant_retrieved) / len(relevant)

def f1_at_k(precision, recall):
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

def average_precision_func(relevant, retrieved):
    if len(relevant) == 0:
        return 0.0
    ap = 0.0
    num_relevant = 0
    for i, doc_id in enumerate(retrieved, start=1):
        if doc_id in relevant:
            num_relevant += 1
            ap += num_relevant / i
    return ap / len(relevant)

def evaluate_query(relevant_ids, retrieved_docs, k_values=[3,6,10]):

    evaluation = {}
    p_10 = precision_at_k(relevant_ids, retrieved_docs, 10)
    r_10 = recall_at_k(relevant_ids, retrieved_docs, 10)
    f1_10 = f1_at_k(p_10, r_10)

    evaluation['Precision'] = p_10
    evaluation['Recall'] = r_10
    evaluation['F1'] = f1_10

    p_3 = precision_at_k(relevant_ids, retrieved_docs, 3)
    p_6 = precision_at_k(relevant_ids, retrieved_docs, 6)
    p_10_specific = precision_at_k(relevant_ids, retrieved_docs, 10)

    evaluation['Precision@3'] = p_3
    evaluation['Precision@6'] = p_6
    evaluation['Precision@10'] = p_10_specific

    ap = average_precision_func(relevant_ids, retrieved_docs)
    evaluation['Average Precision'] = ap

    return evaluation

def retrieve_with_tfidf(data_df, query, ngram_range=(1,1), top_k=10):
    tfidf_vectorizer = TfidfVectorizer(ngram_range=ngram_range)
    tfidf_matrix = tfidf_vectorizer.fit_transform(data_df['Processed_Lyrics'])
    query_vector = tfidf_vectorizer.transform([query])
    cosine_sim = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_indices = cosine_sim.argsort()[-top_k:][::-1]

    retrieved_docs = lyrics_df.iloc[top_indices]['id'].tolist()
    return retrieved_docs

In [ ]:
k_values = [3, 6, 10]
top_k = 10

all_ngram_evaluations = []

for n in range(1, 6):
    evaluation_results = []
    for idx, row in ground_truth_df.iterrows():
        query = row['Processed_Query']
        relevant_ids = set(row['Relevant_Song_IDs'])

        retrieved_docs = retrieve_with_tfidf(lyrics_df, query, ngram_range=(n,n), top_k=top_k)
        evaluation = evaluate_query(relevant_ids, retrieved_docs, k_values)
        evaluation['Query'] = row['query']
        evaluation['N-gram'] = n
        evaluation_results.append(evaluation)

    eval_df = pd.DataFrame(evaluation_results)
    print(f"\nEvaluation Results for {n}-gram:")
    print(eval_df.head())

    macro_results = {
        "Metric": [
            "Precision",
            "Recall",
            "F1",
            "Precision@3",
            "Precision@6",
            "Precision@10",
            "Average Precision"
        ],
        "Macro Average": [
            eval_df['Precision'].mean(),
            eval_df['Recall'].mean(),
            eval_df['F1'].mean(),
            eval_df['Precision@3'].mean(),
            eval_df['Precision@6'].mean(),
            eval_df['Precision@10'].mean(),
            eval_df['Average Precision'].mean()
        ]
    }
    macro_df = pd.DataFrame(macro_results)
    macro_df['N-gram'] = n
    print(f"\nMacro Averages for {n}-gram:")
    print(macro_df)

    eval_df.to_excel(f'evaluation_tfidf_{n}gram.xlsx', index=False)
    macro_df.to_excel(f'macro_averages_{n}gram.xlsx', index=False)

    all_ngram_evaluations.append(macro_df)

combined_macro = pd.concat(all_ngram_evaluations, ignore_index=True)
combined_macro.to_excel('combined_macro_averages_all_ngrams.xlsx', index=False)
print("\nAll macro averages for all n-grams have been saved to 'combined_macro_averages_all_ngrams.xlsx'.")



Evaluation Results for 1-gram:
   Precision    Recall        F1  Precision@3  Precision@6  Precision@10  \
0        0.0  0.000000  0.000000     0.000000     0.000000           0.0   
1        0.0  0.000000  0.000000     0.000000     0.000000           0.0   
2        0.2  0.111111  0.142857     0.333333     0.333333           0.2   
3        0.1  0.250000  0.142857     0.000000     0.000000           0.1   
4        0.1  0.333333  0.153846     0.000000     0.166667           0.1   

   Average Precision          Query  N-gram  
0           0.000000    love you in       1  
1           0.000000   home tonight       1  
2           0.074074  something new       1  
3           0.031250     i held you       1  
4           0.066667  stars tonight       1  

Macro Averages for 1-gram:
              Metric  Macro Average  N-gram
0          Precision       0.088000       1
1             Recall       0.166111       1
2                 F1       0.108785       1
3        Precision@3       0.12

In [ ]:
def rrf_fusion(ranked_lists, rrf_k=60):
    rrf_scores = defaultdict(float)
    for ranked_list in ranked_lists:
        for rank, doc_id in enumerate(ranked_list):
            rrf_scores[doc_id] += 1.0 / (rrf_k + rank + 1)
    fused_ranking = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    fused_docs = [doc_id for doc_id, score in fused_ranking]
    return fused_docs

rrf_evaluations = []

for idx, row in ground_truth_df.iterrows():
    query = row['Processed_Query']
    relevant_ids = set(row['Relevant_Song_IDs'])

    ranked_lists = []
    for n in range(1, 6):
        retrieved_docs = retrieve_with_tfidf(lyrics_df, query, ngram_range=(n,n), top_k=top_k)
        ranked_lists.append(retrieved_docs)

    fused_docs = rrf_fusion(ranked_lists, rrf_k=60)

    rrf_eval = evaluate_query(relevant_ids, fused_docs, k_values)
    rrf_eval['Query'] = row['query']
    rrf_evaluations.append(rrf_eval)

rrf_df = pd.DataFrame(rrf_evaluations)
print("\nRRF Fused Evaluation Results:")
print(rrf_df.head())

rrf_macro_results = {
    "Metric": [
        "Precision",
        "Recall",
        "F1",
        "Precision@3",
        "Precision@6",
        "Precision@10",
        "Average Precision"
    ],
    "Macro Average": [
        rrf_df['Precision'].mean(),
        rrf_df['Recall'].mean(),
        rrf_df['F1'].mean(),
        rrf_df['Precision@3'].mean(),
        rrf_df['Precision@6'].mean(),
        rrf_df['Precision@10'].mean(),
        rrf_df['Average Precision'].mean()
    ]
}
rrf_macro_df = pd.DataFrame(rrf_macro_results)
rrf_macro_df['N-gram'] = 'RRF Fusion'
print("\nMacro Averages for RRF Fusion:")
print(rrf_macro_df)

rrf_df.to_excel('evaluation_tfidf_rrf_fusion.xlsx', index=False)
rrf_macro_df.to_excel('macro_averages_rrf_fusion.xlsx', index=False)
print("\nRRF Fused evaluation results have been saved to 'evaluation_tfidf_rrf_fusion.xlsx'.")
print("Macro averages for RRF Fusion have been saved to 'macro_averages_rrf_fusion.xlsx'.")



RRF Fused Evaluation Results:
   Precision  Recall   F1  Precision@3  Precision@6  Precision@10  \
0        0.0     0.0  0.0          0.0          0.0           0.0   
1        0.0     0.0  0.0          0.0          0.0           0.0   
2        0.0     0.0  0.0          0.0          0.0           0.0   
3        0.0     0.0  0.0          0.0          0.0           0.0   
4        0.0     0.0  0.0          0.0          0.0           0.0   

   Average Precision          Query  
0           0.000000    love you in  
1           0.201063   home tonight  
2           0.149886  something new  
3           0.148310     i held you  
4           0.148252  stars tonight  

Macro Averages for RRF Fusion:
              Metric  Macro Average      N-gram
0          Precision       0.088000  RRF Fusion
1             Recall       0.187857  RRF Fusion
2                 F1       0.115698  RRF Fusion
3        Precision@3       0.106667  RRF Fusion
4        Precision@6       0.073333  RRF Fusion
5     

## Complex

In [ ]:
import pandas as pd
import numpy as np
import re
import warnings
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import math
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings("ignore")

nltk.download('punkt')
nltk.download('punkt_tab')

lyrics_df = pd.read_excel('/content/databersih.xlsx')
ground_truth_df = pd.read_excel('/content/GT UAS NLP_complex.xlsx')

stemmer = PorterStemmer()
remove_words = ["song", "with", "lyrics", "from", "the", "album", "released", "in", "before", "after", "since", "s"]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
def clean_lyrics(text):
    if isinstance(text, str):
        text = re.sub(r'\[.*?\]', '', text)
        text = re.sub(r'\(.*?\)', '', text)
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)
        text = text.lower()
        tokens = word_tokenize(text)
        tokens = [stemmer.stem(word) for word in tokens if word not in remove_words]
        text = ' '.join(tokens)
    return text

lyrics_df['Processed_Lyrics'] = lyrics_df['lyrics'].apply(clean_lyrics)

print("\nCleaned Lyrics Sample:")
print(lyrics_df[['title', 'Processed_Lyrics']].head())

def parse_song_ids(song_id_entry):
    if pd.isnull(song_id_entry):
        return []
    if isinstance(song_id_entry, int):
        return [song_id_entry]
    if isinstance(song_id_entry, float) and np.isnan(song_id_entry):
        return []
    song_id_str = str(song_id_entry)
    return [int(id_.strip()) for id_ in song_id_str.split(',') if id_.strip().isdigit()]

ground_truth_df['Relevant_Song_IDs'] = ground_truth_df['song_id'].apply(parse_song_ids)
ground_truth_df = ground_truth_df[ground_truth_df['Relevant_Song_IDs'].map(len) > 0].reset_index(drop=True)
print("\nParsed Ground Truth:")
print(ground_truth_df[['query', 'Relevant_Song_IDs', 'total']].head())

def preprocess_query(query):
    if isinstance(query, str):
        query = re.sub(r'[^\w\s]', '', query)
        query = re.sub(r'\d+', '', query)
        query = query.lower()
        tokens = word_tokenize(query)
        tokens = [stemmer.stem(word) for word in tokens if word not in remove_words]
        return ' '.join(tokens)
    return query

ground_truth_df['Processed_Query'] = ground_truth_df['query'].apply(preprocess_query)


Cleaned Lyrics Sample:
                   title                                   Processed_Lyrics
0      Chasing Pavements  ive made up my mind dont need to think it over...
1          Cold Shoulder  you say it all my head and thing i think just ...
2         Hometown Glory  ive been walk same way as i did miss out crack...
3  Make You Feel My Love  when rain is blow your face and whole world is...
4                My Same  aye aye ayeay aye aye ayeay aye aye ayeay aye ...

Parsed Ground Truth:
                                               query Relevant_Song_IDs  total
0               Adele's song with lyrics love you in            [1433]      1
1    Billie Eilish's song with lyrics stick together             [227]      1
2              Bruno Mars' song with lyrics kiss you            [2129]      1
3    Jennifer Lopez's song with lyrics you feel left             [788]      1
4  Song released in 2024 with lyrics hopeless rom...            [3019]      1


In [ ]:
# Evaluation Metrics
def precision_at_k(relevant, retrieved, k):
    if k == 0:
        return 0.0
    retrieved_k = retrieved[:k]
    relevant_retrieved = set(retrieved_k).intersection(relevant)
    return len(relevant_retrieved) / k

def recall_at_k(relevant, retrieved, k):
    if len(relevant) == 0:
        return 0.0
    retrieved_k = retrieved[:k]
    relevant_retrieved = set(retrieved_k).intersection(relevant)
    return len(relevant_retrieved) / len(relevant)

def f1_at_k(precision, recall):
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

def average_precision_func(relevant, retrieved):
    if len(relevant) == 0:
        return 0.0
    ap = 0.0
    num_relevant = 0
    for i, doc_id in enumerate(retrieved, start=1):
        if doc_id in relevant:
            num_relevant += 1
            ap += num_relevant / i
    return ap / len(relevant)

def evaluate_query(relevant_ids, retrieved_docs, k_values=[3,6,10]):
    """
    Evaluate a single query's results at given k-values and also compute overall P, R, F1 at k=10.
    """
    evaluation = {}
    p_10 = precision_at_k(relevant_ids, retrieved_docs, 10)
    r_10 = recall_at_k(relevant_ids, retrieved_docs, 10)
    f1_10 = f1_at_k(p_10, r_10)

    evaluation['Precision'] = p_10
    evaluation['Recall'] = r_10
    evaluation['F1'] = f1_10
    p_3 = precision_at_k(relevant_ids, retrieved_docs, 3)
    p_6 = precision_at_k(relevant_ids, retrieved_docs, 6)
    p_10_specific = precision_at_k(relevant_ids, retrieved_docs, 10)

    evaluation['Precision@3'] = p_3
    evaluation['Precision@6'] = p_6
    evaluation['Precision@10'] = p_10_specific

    ap = average_precision_func(relevant_ids, retrieved_docs)
    evaluation['Average Precision'] = ap

    return evaluation

def retrieve_with_tfidf(data_df, query, ngram_range=(1,1), top_k=10):
    tfidf_vectorizer = TfidfVectorizer(ngram_range=ngram_range)
    tfidf_matrix = tfidf_vectorizer.fit_transform(data_df['Processed_Lyrics'])
    query_vector = tfidf_vectorizer.transform([query])
    cosine_sim = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_indices = cosine_sim.argsort()[-top_k:][::-1]

    retrieved_docs = lyrics_df.iloc[top_indices]['id'].tolist()
    return retrieved_docs

In [ ]:
k_values = [3, 6, 10]
top_k = 10

all_ngram_evaluations = []

for n in range(1, 6):
    evaluation_results = []
    for idx, row in ground_truth_df.iterrows():
        query = row['Processed_Query']
        relevant_ids = set(row['Relevant_Song_IDs'])

        retrieved_docs = retrieve_with_tfidf(lyrics_df, query, ngram_range=(n,n), top_k=top_k)
        evaluation = evaluate_query(relevant_ids, retrieved_docs, k_values)
        evaluation['Query'] = row['query']
        evaluation['N-gram'] = n
        evaluation_results.append(evaluation)

    eval_df = pd.DataFrame(evaluation_results)
    print(f"\nEvaluation Results for {n}-gram:")
    print(eval_df.head())

    macro_results = {
        "Metric": [
            "Precision",
            "Recall",
            "F1",
            "Precision@3",
            "Precision@6",
            "Precision@10",
            "Average Precision"
        ],
        "Macro Average": [
            eval_df['Precision'].mean(),
            eval_df['Recall'].mean(),
            eval_df['F1'].mean(),
            eval_df['Precision@3'].mean(),
            eval_df['Precision@6'].mean(),
            eval_df['Precision@10'].mean(),
            eval_df['Average Precision'].mean()
        ]
    }
    macro_df = pd.DataFrame(macro_results)
    macro_df['N-gram'] = n
    print(f"\nMacro Averages for {n}-gram:")
    print(macro_df)

    eval_df.to_excel(f'evaluation_tfidf_{n}gram.xlsx', index=False)
    macro_df.to_excel(f'macro_averages_{n}gram.xlsx', index=False)

    all_ngram_evaluations.append(macro_df)

combined_macro = pd.concat(all_ngram_evaluations, ignore_index=True)
combined_macro.to_excel('combined_macro_averages_all_ngrams.xlsx', index=False)
print("\nAll macro averages for all n-grams have been saved to 'combined_macro_averages_all_ngrams.xlsx'.")



Evaluation Results for 1-gram:
   Precision  Recall        F1  Precision@3  Precision@6  Precision@10  \
0        0.0     0.0  0.000000     0.000000     0.000000           0.0   
1        0.1     1.0  0.181818     0.333333     0.166667           0.1   
2        0.0     0.0  0.000000     0.000000     0.000000           0.0   
3        0.0     0.0  0.000000     0.000000     0.000000           0.0   
4        0.1     1.0  0.181818     0.000000     0.166667           0.1   

   Average Precision                                              Query  \
0           0.000000               Adele's song with lyrics love you in   
1           0.333333    Billie Eilish's song with lyrics stick together   
2           0.000000              Bruno Mars' song with lyrics kiss you   
3           0.000000    Jennifer Lopez's song with lyrics you feel left   
4           0.166667  Song released in 2024 with lyrics hopeless rom...   

   N-gram  
0       1  
1       1  
2       1  
3       1  
4       1  


In [ ]:
def rrf_fusion(ranked_lists, rrf_k=60):
    rrf_scores = defaultdict(float)
    for ranked_list in ranked_lists:
        for rank, doc_id in enumerate(ranked_list):
            rrf_scores[doc_id] += 1.0 / (rrf_k + rank + 1)
    fused_ranking = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    fused_docs = [doc_id for doc_id, score in fused_ranking]
    return fused_docs

rrf_evaluations = []

for idx, row in ground_truth_df.iterrows():
    query = row['Processed_Query']
    relevant_ids = set(row['Relevant_Song_IDs'])

    ranked_lists = []
    for n in range(1, 6):
        retrieved_docs = retrieve_with_tfidf(lyrics_df, query, ngram_range=(n,n), top_k=top_k)
        ranked_lists.append(retrieved_docs)

    fused_docs = rrf_fusion(ranked_lists, rrf_k=60)

    rrf_eval = evaluate_query(relevant_ids, fused_docs, k_values)
    rrf_eval['Query'] = row['query']
    rrf_evaluations.append(rrf_eval)

rrf_df = pd.DataFrame(rrf_evaluations)
print("\nRRF Fused Evaluation Results:")
print(rrf_df.head())

rrf_macro_results = {
    "Metric": [
        "Precision",
        "Recall",
        "F1",
        "Precision@3",
        "Precision@6",
        "Precision@10",
        "Average Precision"
    ],
    "Macro Average": [
        rrf_df['Precision'].mean(),
        rrf_df['Recall'].mean(),
        rrf_df['F1'].mean(),
        rrf_df['Precision@3'].mean(),
        rrf_df['Precision@6'].mean(),
        rrf_df['Precision@10'].mean(),
        rrf_df['Average Precision'].mean()
    ]
}
rrf_macro_df = pd.DataFrame(rrf_macro_results)
rrf_macro_df['N-gram'] = 'RRF Fusion'
print("\nMacro Averages for RRF Fusion:")
print(rrf_macro_df)

rrf_df.to_excel('evaluation_tfidf_rrf_fusion.xlsx', index=False)
rrf_macro_df.to_excel('macro_averages_rrf_fusion.xlsx', index=False)
print("\nRRF Fused evaluation results have been saved to 'evaluation_tfidf_rrf_fusion.xlsx'.")
print("Macro averages for RRF Fusion have been saved to 'macro_averages_rrf_fusion.xlsx'.")



RRF Fused Evaluation Results:
   Precision  Recall   F1  Precision@3  Precision@6  Precision@10  \
0        0.0     0.0  0.0          0.0          0.0           0.0   
1        0.0     0.0  0.0          0.0          0.0           0.0   
2        0.0     0.0  0.0          0.0          0.0           0.0   
3        0.0     0.0  0.0          0.0          0.0           0.0   
4        0.0     0.0  0.0          0.0          0.0           0.0   

   Average Precision                                              Query  
0           0.000000               Adele's song with lyrics love you in  
1           0.083333    Billie Eilish's song with lyrics stick together  
2           0.000000              Bruno Mars' song with lyrics kiss you  
3           0.076923    Jennifer Lopez's song with lyrics you feel left  
4           0.083333  Song released in 2024 with lyrics hopeless rom...  

Macro Averages for RRF Fusion:
              Metric  Macro Average      N-gram
0          Precision       0.0